In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.alert import Alert
import re
from selenium.webdriver.support.ui import Select
from time import sleep

In [2]:
dir_for_shinhan = 'C:/Users/shic/Desktop/shinhan_simon/chromedriver/chromedriver.exe'
dir_for_seunghun = 'C:/Users/rsh15/Desktop/seunghuni/selenium/chromedriver.exe'

In [3]:
# chrome driver 지정해주고, 열어서 법원경매 사이트 접속
driver = webdriver.Chrome(dir_for_shinhan)
driver.get(url = 'http://www.courtauction.go.kr/RealUtilMaeTong.laf')

# 시/도 옵션 목록 가져오고, 리스트로 만들기
soup = BeautifulSoup(driver.page_source,'html.parser')
big_regions = soup.find('select',{'id':'idSidoCode'}).find_all('option',{'value':re.compile('\d')})
big_regions = [k.text for k in big_regions]

# 소재지 버튼 클릭
driver.find_element_by_id('idBubwLocGubun2').click()

# 창 maximize
driver.maximize_window()

In [4]:
big_regions

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

각 시/도의 시/군/구별로 월별 데이터를 가져옴.   
개별 데이터를 가져오는 것은 시간이 오래 걸리지 않으나, 경우의 수가 축적되며 상당히 오랜 시간이 걸림

In [5]:
regions_final = []
for sido in ['서울특별시','경기도','인천광역시']:
    print(sido,'start.')
    # 한 시/도의 데이터프레임을 하나의 리스트에 담는다.
    big_regions_list = []
    
    # 시/도 옵션 입력란 찾아서 넣어준다.
    select1= driver.find_element_by_id('idSidoCode')
    select1.send_keys(sido)
    
    # 시/도 옵션에 따라 바뀌는 시/군/구 데이터
    soup = BeautifulSoup(driver.page_source,'html.parser')
    small_regions = soup.find('select',{'id':'idSiguCode'}).find_all('option',{'value':re.compile('\d')})
    small_regions = [k.text.replace('\n','') for k in small_regions]
    
    # 시/도 마다 시/군/구 데이터로 Loop
    for sigu in small_regions:
        select_sido = driver.find_element_by_id('idSidoCode')
        select_sido.send_keys(sido)
        
        select_sigu = driver.find_element_by_id('idSiguCode')
        select_sigu.send_keys(sigu)
        
        for year in range(2017,2022):
            for month in range(1,13):
                if (year==2021 and month > 6):
                    break
                
                # send start year_month
                select_start_year = driver.find_element_by_id('idSelStartYear')
                select_start_year.send_keys(str(year))
                
                select_start_month = driver.find_element_by_id('idSelStartMnth')
                select_start_month.send_keys(str(month))

                # send enddd year_month
                select_enddd_year = driver.find_element_by_id('idSelEndYear')
                select_enddd_year.send_keys(str(year))
                
                select_enddd_month = driver.find_element_by_id('idSelEndMnth')
                select_enddd_month.send_keys(str(month))
                
                # 조회버튼 클릭
                driver.find_element_by_class_name('Ltbl_srch_btn').click()

                # page_source 가져와서 테이블 파싱
                soup = BeautifulSoup(driver.page_source,'html.parser')
                parse_table = pd.read_html(driver.page_source)[0]

                # parse_table에 시/도, 시/군/구 컬럼 추가
                parse_table['sido'] = sido
                parse_table['sigu'] = sigu
                parse_table['year'] = year
                
                # 날짜 형식(월) 일원화
                if len(str(month)) == 1:
                    parse_table['month'] = '0'+str(month)
                else:
                    parse_table['month'] = str(month)
        
                # 시/도 리스트에 데이터프레임 추가
                big_regions_list.append(parse_table)
    print(sido,'complete.',len(small_regions),'small countries.')
    
    # 한 시/도 Loop가 끝나면 최종 리스트에 추가
    regions_final.append(big_regions_list)

서울특별시 start.
서울특별시 complete. 25 small countries
경기도 start.
경기도 complete. 58 small countries
인천광역시 start.
인천광역시 complete. 13 small countries


In [16]:
df_seoul   = pd.concat(regions_final[0],axis=0)
df_gyeongi = pd.concat(regions_final[1],axis=0)
df_incheon = pd.concat(regions_final[2],axis=0)

In [17]:
df_court = pd.concat([df_seoul,df_gyeongi,df_incheon],axis=0)

In [18]:
# save data to csv
dirr = 'C:/Users/shic/Desktop/crawler_project/data_save/courtauction'
df_court.to_csv(dirr+'/df_court.csv',index=False)

In [19]:
df_court

,물건용도,물건용도.1,경매건수,매각건수,감정가(단위:원),매각가(단위:원),매각율,매각가율,sido,sigu,year,month
0,아파트,아파트,7,7,9560988000,9163107000,100%,95.8%,서울특별시,강남구,2017,01
1,"단독주택,다가구주택",단독주택,0,0,0,0,0%,0%,서울특별시,강남구,2017,01
2,"단독주택,다가구주택",다가구주택,0,0,0,0,0%,0%,서울특별시,강남구,2017,01
3,"단독주택,다가구주택",겸용,0,0,0,0,0%,0%,서울특별시,강남구,2017,01
4,"단독주택,다가구주택",소계,0,0,0,0,0%,0%,서울특별시,강남구,2017,01
...,...,...,...,...,...,...,...,...,...,...,...,...
14,"상가,오피스텔,근린시설",근린시설,5,2,2139816220,2151000011,40%,100.5%,인천광역시,중구,2021,06
15,"상가,오피스텔,근린시설",겸용,2,1,1478000000,850000000,50%,57.5%,인천광역시,중구,2021,06
16,"상가,오피스텔,근린시설",소계,11,3,3617816220,3001000011,27.3%,83%,인천광역시,중구,2021,06
17,기타,기타,0,0,0,0,0%,0%,인천광역시,중구,2021,06
